In [1]:
import sys
import os
import open3d as o3d
import numpy as np

SCRIPT_DIR = os.getcwd()
REPO_DIR = os.path.dirname(SCRIPT_DIR)
sys.path.append(REPO_DIR)

from utils import FileManager
from frame import Frame
from visualizer import generic_o3d_vis
from world import World

RUN_DIR_PATH = os.path.join(REPO_DIR, "sample_data", "run_01")

WIDTH = 848
HEIGHT = 480
FOCAL_LENGTH_X = 422.068
FOCAL_LENGTH_Y = 424.824
PRINCIPAL_POINT_X = 404.892
PRINCIPAL_POINT_Y = 260.621
camera_matrix = np.array([
    [FOCAL_LENGTH_X, 0, PRINCIPAL_POINT_X],
    [0, FOCAL_LENGTH_Y, PRINCIPAL_POINT_Y],
    [0, 0, 1]
], dtype=np.float32)


if __name__ == "__main__":
    fm = FileManager(RUN_DIR_PATH)
    world = World()

    start_img_idx = 0
    end_img_idx = 5

    num_frames = end_img_idx - start_img_idx

    total_fill_rate = 0
    total_registered_rate = 0
    total_foreground_rate = 0
    total_background_rate = 0
    total_fg_filtered_rate = 0
    total_bg_filtered_rate = 0

    for idx, pair in enumerate(fm.paired_files[start_img_idx:end_img_idx], start=0):
        frame = Frame(
            frame_idx=idx,
            depth_path=pair.depth_path,
            color_path=pair.color_path,
            raw_timestamp=pair.raw_timestamp,
            recording_time_str=pair.recording_time_str
        )

        pcd = frame._filter_pixels(199, 149).create_depth_corrected_pcd()

        total_pixels = frame.raw_depth_img.size
        total_valid_pixels = np.count_nonzero(frame.raw_depth_img)
        fill_rate = total_valid_pixels / total_pixels

        foreground_pixels = frame.all_pixels.get_foreground_pixels()
        background_pixels = frame.all_pixels.get_background_pixels()

        foreground_count = len(foreground_pixels)
        background_count = len(background_pixels)
        registered_count = len(pcd.points)

        foreground_rate = foreground_count / total_valid_pixels if total_valid_pixels > 0 else 0
        background_rate = background_count / total_valid_pixels if total_valid_pixels > 0 else 0
        registered_rate = registered_count / total_valid_pixels if total_valid_pixels > 0 else 0

        foreground_rate_total = foreground_count / total_pixels
        background_rate_total = background_count / total_pixels
        registered_rate_total = registered_count / total_pixels

        bg_uvs = set((p.u, p.v) for p in background_pixels.pixels)
        fg_uvs = set((p.u, p.v) for p in foreground_pixels.pixels)

        filtered_uvs = set(
            (int(round(point[0] * FOCAL_LENGTH_X / point[2] + PRINCIPAL_POINT_X)),
             int(round(point[1] * FOCAL_LENGTH_Y / point[2] + PRINCIPAL_POINT_Y)))
            for point in np.asarray(pcd.points)
        )

        bg_retained = len(bg_uvs & filtered_uvs)
        fg_retained = len(fg_uvs & filtered_uvs)

        bg_filtered_out = background_count - bg_retained
        fg_filtered_out = foreground_count - fg_retained

        bg_filtered_rate = bg_filtered_out / background_count if background_count > 0 else 0
        fg_filtered_rate = fg_filtered_out / foreground_count if foreground_count > 0 else 0

        total_fill_rate += fill_rate
        total_registered_rate += registered_rate
        total_foreground_rate += foreground_rate
        total_background_rate += background_rate
        total_fg_filtered_rate += fg_filtered_rate
        total_bg_filtered_rate += bg_filtered_rate

        print(f"----- Frame {idx} -----")
        print(f"Fill Rate           : {fill_rate*100:.1f}% ({total_valid_pixels} / {total_pixels})")
        print(f"Registered Rate     : {registered_rate*100:.1f}% ({registered_count} / {total_valid_pixels}) [{registered_rate_total*100:.1f}% of total]")
        print("-" * 20)
        print(f"Foreground Rate     : {foreground_rate*100:.1f}% ({foreground_count} / {total_valid_pixels}) [{foreground_rate_total*100:.1f}% of total]")
        print(f"Foreground filtered : {fg_filtered_rate*100:.1f}% ({fg_filtered_out} / {foreground_count})")
        print("-" * 20)
        print(f"Background Rate     : {background_rate*100:.1f}% ({background_count} / {total_valid_pixels}) [{background_rate_total*100:.1f}% of total]")
        print(f"Background filtered : {bg_filtered_rate*100:.1f}% ({bg_filtered_out} / {background_count})")
        print("-" * 20)
        print("")

    if num_frames > 0:
        avg_fill_rate = total_fill_rate / num_frames
        avg_registered_rate = total_registered_rate / num_frames
        avg_foreground_rate = total_foreground_rate / num_frames
        avg_background_rate = total_background_rate / num_frames
        avg_fg_filtered_rate = total_fg_filtered_rate / num_frames
        avg_bg_filtered_rate = total_bg_filtered_rate / num_frames

        print("===== AVERAGE METRICS OVER ALL FRAMES =====")
        print(f"Fill Rate           : {avg_fill_rate*100:.1f}%")
        print(f"Registered Rate     : {avg_registered_rate*100:.1f}%")
        print("-" * 20)
        print(f"Foreground Rate     : {avg_foreground_rate*100:.1f}%")
        print(f"Foreground filtered : {avg_fg_filtered_rate*100:.1f}%")
        print("-" * 20)
        print(f"Background Rate     : {avg_background_rate*100:.1f}%")
        print(f"Background filtered : {avg_bg_filtered_rate*100:.1f}%")
        print("-" * 20)
        print("")




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
----- Frame 0 -----
Fill Rate           : 18.0% (73284 / 407040)
Registered Rate     : 27.1% (19886 / 73284) [4.9% of total]
--------------------
Foreground Rate     : 59.1% (43342 / 73284) [10.6% of total]
Foreground filtered : 75.6% (32766 / 43342)
--------------------
Background Rate     : 40.9% (29942 / 73284) [7.4% of total]
Background filtered : 68.9% (20632 / 29942)
--------------------

----- Frame 1 -----
Fill Rate           : 18.9% (76982 / 407040)
Registered Rate     : 27.0% (20802 / 76982) [5.1% of total]
--------------------
Foreground Rate     : 57.5% (44296 / 76982) [10.9% of total]
Foreground filtered : 75.8% (33586 / 44296)
--------------------
Background Rate     : 42.5% (32686 / 76982) [8.0% of total]
Background filtered : 69.1% (22594 / 32686)
--------------------

----- Frame 2 -----
Fill Rate      